## Import Req'd Libraries 

In [308]:
import pandas as pd
import os
import seaborn as sns

In [309]:
#Getting all the file_names in a given directory
def get_file_names(folder):
    #Listing entries present in given folder
    entries = os.listdir(folder)
    for i in entries:
        if 'csv' not in i:
            entries.remove(i)
    return sorted(entries, reverse=True)

#Saving the file
def save_file(root, name_of_file, my_dataframe):
    #Test if save directory exists
    try:
        my_dataframe.to_csv(root+'processed_data/'+ name_of_file, index=False)
    #Otherwise make the directory and then save
    except:
        os.mkdir(root+'processed_data')
        my_dataframe.to_csv(root+'processed_data/'+ name_of_file, index=False)

In [310]:
def initialization(filepath):
    df = pd.read_csv(filepath, 
                     usecols = ['id','last_scraped', 'host_is_superhost', 
                            'latitude','longitude', 'property_type',
                           'room_type','accommodates','bathrooms',
                           'bedrooms','beds','amenities', 'price',
                            'instant_bookable','cancellation_policy'])
    df["last_scraped"] = pd.to_datetime(df["last_scraped"])
    df["price"] = df["price"].apply(lambda x: x.replace('$','').replace(',', '').replace('.00', '')).astype("int")
    return df

## Concatenating All Data Together

In [311]:
# get list of filenames
root = '../'
name = get_file_names(root+'original_dataset')
accum_kept = 0
accum_removed = 0

# combine all files into one
print("COMBINING INTO ONE DATAFRAME...")
frames = []
for i in name:
    print("--Processing: ", i)
    filepath = root+'original_dataset/'+i
    frames.append(initialization(filepath))
full_df = pd.concat(frames, sort=False)

initial_samples = len(full_df)
print("Initial # of Samples: ", len(full_df))

print(" ")

# Drop missing values
print("DELETING SAMPLES WITH MISSING VALUES...")
full_df = full_df.dropna()
samples1 = len(full_df)
print("--Number of Samples Removed: ", initial_samples-samples1)

# reset index
full_df = full_df.reset_index(drop=True)

COMBINING INTO ONE DATAFRAME...
--Processing:  19_08_08_listings.csv
--Processing:  19_07_08_listings.csv
--Processing:  19_06_04_listings.csv
--Processing:  19_05_06_listings.csv
--Processing:  19_04_08_listings.csv
--Processing:  19_03_07_listings.csv
--Processing:  19_02_04_listings.csv
--Processing:  19_01_13_listings.csv
--Processing:  18_12_06_listings.csv
--Processing:  18_11_04_listings.csv
--Processing:  18_10_06_listings.csv
--Processing:  18_09_08_listings.csv
Initial # of Samples:  236507
 
DELETING SAMPLES WITH MISSING VALUES...
--Number of Samples Removed:  559


## Numericalize Categorical Data

#### Mapping for Categorical Features

In [312]:
cancellation_policies = {
    "flexible": 1,
    "moderate": 5,
    "strict_14_with_grace_period": 14,
    "super_strict_30": 30,
    "super_strict_60": 60
}
room_types = {
    "Entire home/apt": 0,
    "Private room": 1,
    "Shared room": 2
}
true_false = {
    't': 1,
    'f': 0
}

In [313]:
# convert t/f into 1/0
print("NUMERICALIZING TRUE/FALSE...")
for i in ["host_is_superhost", "instant_bookable"]:
    full_df[i] = full_df[i].map(true_false)
    print(full_df[i].value_counts())

print(" ")

# cancellation policy
print("NUMERICALIZING CANCELLATION POLICY...")
print(full_df["cancellation_policy"].value_counts())
full_df["cancellation_policy"] = full_df["cancellation_policy"].map(cancellation_policies)

print(" ")

# room_type
print("NUMERICALIZING ROOM TYPE...")
print(full_df["room_type"].value_counts())
full_df["room_type"] = full_df["room_type"].map(room_types)


NUMERICALIZING TRUE/FALSE...
0    175881
1     60067
Name: host_is_superhost, dtype: int64
0    147446
1     88502
Name: instant_bookable, dtype: int64
 
NUMERICALIZING CANCELLATION POLICY...
strict_14_with_grace_period    93976
moderate                       70987
flexible                       70497
super_strict_30                  345
super_strict_60                  143
Name: cancellation_policy, dtype: int64
 
NUMERICALIZING ROOM TYPE...
Entire home/apt    152229
Private room        79583
Shared room          4136
Name: room_type, dtype: int64


#### Mapping for Property Types

In [314]:
property_types = {
    "House" : 1,
    "Apartment" : 2,
    "Condominium" : 5,
    "Townhouse" : 3,
    "Bungalow" : 0,
    "Loft" : 4,
    "Serviced Apartment" : 6,
    "Guest Suite" : 7
}

In [315]:
print("FILTERLING AND NUMERICALIZING PROPERTY TYPES...")
property_type_counts = full_df["property_type"].value_counts()
accum = []
for i in range(len(property_type_counts)):
    if property_type_counts[i] >= 0.01*samples1:
        accum.append(property_type_counts.index[i])
print("Types of Included Properties:")
print(accum)

full_df = full_df[full_df["property_type"].isin(accum)]
print(full_df["property_type"].value_counts())

full_df["property_type"] = full_df["property_type"].map(property_types)

samples2 = len(full_df)
print("Number of Samples Removed:", samples1 - samples2)

FILTERLING AND NUMERICALIZING PROPERTY TYPES...
Types of Included Properties:
['Apartment', 'House', 'Condominium', 'Townhouse', 'Guest suite', 'Bungalow', 'Loft', 'Serviced apartment']
Apartment             73132
House                 62791
Condominium           59341
Townhouse             11401
Guest suite            8561
Bungalow               7278
Loft                   5268
Serviced apartment     2676
Name: property_type, dtype: int64
Number of Samples Removed: 5500


### Store Mean and Std Dev

In [316]:
mean_std = pd.DataFrame(columns = ["mean", "std"])

### Normalizing Price

In [317]:
print("(BEFORE) PRICE STATISTICS...")
print(full_df["price"].describe())
print(full_df["price"].quantile([0.01, 0.05, 0.1, 0.9, 0.95, 0.99]))

print("")
print("TRIMMING PRICE OUTLIERS...")
min_price = full_df["price"].quantile(0.05)
max_price = full_df["price"].quantile(0.95)

full_df = full_df[(full_df["price"] >= min_price) & (full_df["price"] <= max_price)]
samples3 = len(full_df)
print("--Number of Samples Removed: ", samples2-samples3)

print("")
print("STANDARDIZING PRICE...") # using Z-score
mean_price = full_df["price"].mean()
std_price = full_df["price"].std()
full_df["price"] = (full_df["price"] - mean_price)/std_price
print("--Mean Price: ", mean_price)
print("--Std Price: ", std_price)
temp = pd.DataFrame([[mean_price, std_price]], columns=["mean", "std"], index=["price"])
mean_std = mean_std.append(temp)

(BEFORE) PRICE STATISTICS...
count    230448.000000
mean        141.131253
std         222.025238
min           0.000000
25%          65.000000
50%         100.000000
75%         160.000000
max       13426.000000
Name: price, dtype: float64
0.01     25.0
0.05     35.0
0.10     43.0
0.90    250.0
0.95    350.0
0.99    700.0
Name: price, dtype: float64

TRIMMING PRICE OUTLIERS...
--Number of Samples Removed:  20833

STANDARDIZING PRICE...
--Mean Price:  120.09710182954464
--Std Price:  68.1989415678001


In [319]:
full_df.head(5)

,id,last_scraped,host_is_superhost,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,instant_bookable,cancellation_policy
1,8077,2019-08-09,0,43.64105,-79.37628,2.0,1,2,1.5,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",-0.294683,1,14
2,12604,2019-08-08,0,43.66724,-79.41598,1.0,1,1,1.5,1.0,1.0,"{Internet,Wifi,""Pets live on this property"",Ca...",-0.778562,0,5
3,23691,2019-08-09,1,43.69602,-79.45468,1.0,1,3,1.0,1.0,1.0,"{Internet,Wifi,""Air conditioning"",Kitchen,""Fre...",-0.705247,1,14
4,26654,2019-08-09,0,43.64530,-79.38940,5.0,0,5,1.0,1.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",1.156952,0,14
5,27423,2019-08-08,0,43.66890,-79.32592,2.0,0,1,1.0,0.0,1.0,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",-0.954518,0,14


### Standardizing Numerical Features


In [320]:
for i in ["latitude", "longitude", "property_type", "room_type", "accommodates", "bathrooms", "bedrooms", "beds", "cancellation_policy"]:
    print("STANDARDIZING "+i.upper()+"...")
    mean = full_df[i].mean()
    std = full_df[i].std()
    temp = pd.DataFrame([[mean, std]], columns=["mean", "std"], index=[i])
    mean_std = mean_std.append(temp)
    full_df[i] = (full_df[i] - mean)/std
    print("--Mean: ", mean)
    print("--Std: ", std)
    print("")

STANDARDIZING LATITUDE...
--Mean:  43.6774921041365
--Std:  0.04610974980603515

STANDARDIZING LONGITUDE...
--Mean:  -79.398326263056
--Std:  0.059359153647078375

STANDARDIZING PROPERTY_TYPE...
--Mean:  2.602684638072134
--Std:  1.635305987842257

STANDARDIZING ROOM_TYPE...
--Mean:  0.340686496672471
--Std:  0.49595747276278074

STANDARDIZING ACCOMMODATES...
--Mean:  3.053679364549293
--Std:  1.762131516737716

STANDARDIZING BATHROOMS...
--Mean:  1.200374496099993
--Std:  0.45931966657010054

STANDARDIZING BEDROOMS...
--Mean:  1.2703432483362356
--Std:  0.7651023775852693

STANDARDIZING BEDS...
--Mean:  1.6253655511294516
--Std:  0.9610655274398833

STANDARDIZING CANCELLATION_POLICY...
--Mean:  7.400939818238198
--Std:  5.685699314132479



In [321]:
corr = full_df 

id                     0.015666
host_is_superhost      0.001140
latitude              -0.119640
longitude              0.003486
room_type             -0.218910
accommodates           0.324023
bathrooms              0.298521
bedrooms               0.295860
beds                   0.286214
price                  1.000000
instant_bookable      -0.021342
cancellation_policy    0.066762
dtype: float64

In [121]:
full_df.head(5)

,id,last_scraped,host_is_superhost,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,instant_bookable,cancellation_policy
1,8077,2019-08-09,0,-0.793778,0.366974,Apartment,0.5,-0.589435,0.633329,-0.349118,-0.644848,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",-0.288464,1,1.077087
2,12604,2019-08-08,0,-0.229058,-0.299423,House,0.5,-1.156138,0.633329,-0.349118,-0.644848,"{Internet,Wifi,""Pets live on this property"",Ca...",-0.773326,0,-0.402349
3,23691,2019-08-09,1,0.391509,-0.949034,House,0.5,-0.022733,-0.433057,-0.349118,-0.644848,"{Internet,Wifi,""Air conditioning"",Kitchen,""Fre...",-0.699862,1,1.077087
4,26654,2019-08-09,0,-0.702138,0.146744,Condominium,0.0,1.110672,-0.433057,-0.349118,0.393678,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",1.166124,0,1.077087
5,27423,2019-08-08,0,-0.193264,1.212308,Apartment,0.0,-1.156138,-0.433057,-1.656324,-0.644848,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",-0.949640,0,1.077087
